# RAG-LLMs

## Setup

### Imports

In [1]:
# imports
import os
from dotenv import load_dotenv
from src.utils import setup_azure_openai
from src.data_utils import SQLDBManager
import ast

%load_ext autoreload
%autoreload 2

### Config

In [49]:
# set openai api key / ad token
# setup azure openai AD token
setup_azure_openai()
# print(f"AD token set: {os.environ['AZURE_OPENAI_AD_TOKEN']}")

Successfully setup Azure OpenAI authentication


## Prepare data

In [44]:
# set db variables
data_table = 'pegadata.ppm_work'
req_cols_path = 'data/pega-as-clone/req_fields.txt'
new_data_table = 'pegadata.ppm_work_filtered'
primary_key = 'pxinsname'
text_col = 'pydescription'

In [45]:
# instantiate sql db manager & connect to sql db
sqldb_manager = SQLDBManager.from_env()

# filter 'ppm_work' table and create new (if not exists) 
sqldb_manager.filter_table(data_table, req_cols_path, new_data_table, primary_key)

connected to database successfully.
table pegadata.ppm_work_filtered created successfully.


In [47]:
# clean text in 'pydescription' before creating embs
cols_to_clean = [text_col]
sqldb_manager.clean_html(new_data_table, cols_to_clean, primary_key)

# print 1st 5 cleaned rows
rows_str = sqldb_manager.db.run(f"SELECT {text_col} FROM {new_data_table} LIMIT 5")
rows = ast.literal_eval(rows_str)  # convert string to list
rows = [row[0] for row in rows]  # convert list of tuples to list of strings
for row in rows:
    print(row)

As PO, I would like to release the first version of GenAI Hub to Staging and
be ready to release into Prod Adoption and Production


NULL
## Details

Description

As PO I would like to test end to end if...


plugin for Agile Studio


<p lang="EN-US"...




In [ ]:
# create embeddings
from langchain.embeddings import AzureOpenAIEmbeddings
embs_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002") # instantiate embeddings model
sqldb_manager.create_embs_col(new_data_table, text_col, embs_model) # create embeddings col

# check embs col
rows_str = sqldb_manager.db.run(f"SELECT {text_col}_embs FROM {new_data_table} LIMIT 5")
rows = ast.literal_eval(rows_str)  # convert string to list
rows = [row[0] for row in rows]  # convert list of tuples to list embs
# print 1st 5 embs
# for row in rows:
#     print(row[:10])